In [1]:
#import
'''
Developed by Abdullah Al Mamun
- Feature selection framework 
- Date: Feb 2019
'''

# Load pkgses
import os
import time
from datetime import datetime
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC 
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn.utils import shuffle
from sklearn.naive_bayes import GaussianNB 
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.linear_model import Lasso
from sklearn import metrics
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix
# Masrur confusion matrix
import collections
import seaborn as sns
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from pylab import savefig

# STG comment if local
# from stg import STG
# import scipy.stats # for creating a simple dataset 
# import torch

#keep comment otherwise fig will be distorted
# %matplotlib inline
# import seaborn as sns
# import seaborn as sns; sns.set(color_codes=True)
# from pylab import savefig


PATH = ''
fileName = 'Example-dataset.csv'

In [2]:
# load data
#     loading data, minMax normalization and train tesing split
df = pd.read_csv(PATH + fileName)
df.fillna(0.0)
# df = df.replace(np.nan,0.0)
X = df.iloc[:,1:-1]
y = df.iloc[:,-1]
df.head()

,ENSG00000005206.15,ENSG00000083622.8,ENSG00000088970.14,ENSG00000099869.7,ENSG00000100181.20,ENSG00000104691.13,ENSG00000115934.11,ENSG00000117242.7,ENSG00000118412.11,ENSG00000122043.9,...,ENSG00000225138.6,ENSG00000225140.1,ENSG00000225144.2,ENSG00000225146.1,ENSG00000225148.1,ENSG00000225152.1,ENSG00000225156.2,ENSG00000225163.3,ENSG00000225166.1,target
0,4.474815,1.015497,3.938578,0.011318,0.217501,2.445204,0.024322,1.651243,1.495478,0.0,...,2.775220,0.038819,0.0,0.059680,0.0,0.14878,0.01209,0.265130,0.0,0
1,4.313495,0.608207,2.179453,0.000000,0.015612,1.441505,0.000000,0.938922,1.537811,0.0,...,5.017635,0.018949,0.0,0.057897,0.0,0.07404,0.00000,0.257730,0.0,0
2,4.396907,1.475637,2.591849,0.016740,0.023794,2.690200,0.000000,1.583583,1.415676,0.0,...,3.744041,0.000000,0.0,0.000000,0.0,0.00000,0.00000,0.377362,0.0,0
3,3.960528,1.142912,2.312048,0.010347,0.043719,2.203999,0.000000,0.965548,1.920312,0.0,...,4.571482,0.000000,0.0,0.107283,0.0,0.00000,0.00000,1.280320,0.0,0
4,4.296875,0.447136,2.104808,0.162264,0.178781,1.747109,0.000000,0.762944,1.715874,0.0,...,2.415390,0.132493,0.0,0.117850,0.0,0.00000,0.00000,0.686951,0.0,0


In [3]:
print(X.shape)
cls = y.unique()
len(cls)

(199, 1022)


2

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, stratify = y, random_state = 1)
print(X_train.shape, X_test.shape)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, train_size=0.8, stratify = y_train, random_state = 1)
print(X_train.shape, X_valid.shape)

(159, 1022) (40, 1022)
(127, 1022) (32, 1022)


In [5]:
# 1. utils.py
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np 
import six
import collections
import copy
from torch.utils.data import Dataset
from collections import defaultdict
import h5py
from scipy.stats import norm
import os

SKIP_TYPES = six.string_types


class SimpleDataset(Dataset):
    '''
    Assuming X and y are numpy arrays and 
     with X.shape = (n_samples, n_features) 
          y.shape = (n_samples,)
    '''
    def __init__(self, X, y=None):
        self.X = X
        self.y = y
    
    def __len__(self):
        return (len(self.X))

    def __getitem__(self, i):
        data = self.X[i]
        data = np.array(data).astype(np.float32)
        if self.y is not None:
            return dict(input=data, label=self.y[i])
        else:
            return dict(input=data)


class FastTensorDataLoader:
    """
    A DataLoader-like object for a set of tensors that can be much faster than
    TensorDataset + DataLoader because dataloader grabs individual indices of
    the dataset and calls cat (slow).
    Source: https://discuss.pytorch.org/t/dataloader-much-slower-than-manual-batching/27014/6
    """
    def __init__(self, *tensors, tensor_names, batch_size=32, shuffle=False):
        """
        Initialize a FastTensorDataLoader.
        :param *tensors: tensors to store. Must have the same length @ dim 0.
        :param tensor_names: name of tensors (for feed_dict)
        :param batch_size: batch size to load.
        :param shuffle: if True, shuffle the data *in-place* whenever an
            iterator is created out of this object.
        :returns: A FastTensorDataLoader.
        """
        assert all(t.shape[0] == tensors[0].shape[0] for t in tensors)
        self.tensors = tensors
        self.tensor_names = tensor_names

        self.dataset_len = self.tensors[0].shape[0]
        self.batch_size = batch_size
        self.shuffle = shuffle

        # Calculate # batches
        n_batches, remainder = divmod(self.dataset_len, self.batch_size)
        if remainder > 0:
            n_batches += 1
        self.n_batches = n_batches

    def __iter__(self):
        if self.shuffle:
            r = torch.randperm(self.dataset_len)
            self.tensors = [t[r] for t in self.tensors]
        self.i = 0
        return self

    def __next__(self):
        if self.i >= self.dataset_len:
            raise StopIteration
        batch = {}
        for k in range(len(self.tensor_names)):
            batch.update({self.tensor_names[k]: self.tensors[k][self.i:self.i+self.batch_size]})
        self.i += self.batch_size
        return batch
        

    def __len__(self):
        return self.n_batches


'''standardize_dataset function is from utils_jared.py'''
def standardize_dataset(dataset, offset, scale):
    norm_ds = copy.deepcopy(dataset)
    norm_ds['x'] = (norm_ds['x'] - offset) / scale
    return norm_ds


'''load_datasets function is from utils_jared.py'''
def load_datasets(dataset_file):
    datasets = defaultdict(dict)
    with h5py.File(dataset_file, 'r') as fp:
        for ds in fp:
            for array in fp[ds]:
                datasets[ds][array] = fp[ds][array][:]

    return datasets

def load_cox_gaussian_data():
    dataset_file = os.path.join(os.path.dirname(__file__), 
        'datasets/gaussian_survival_data.h5')
    datasets = defaultdict(dict)
    with h5py.File(dataset_file, 'r') as fp:
        for ds in fp:
            for array in fp[ds]:
                datasets[ds][array] = fp[ds][array][:]

    return datasets

def prepare_data(x, label):
    if isinstance(label, dict):
       e, t = label['e'], label['t']

    # Sort training data for accurate partial likelihood calculation.
    sort_idx = np.argsort(t)[::-1]
    x = x[sort_idx]
    e = e[sort_idx]
    t = t[sort_idx]

    #return x, {'e': e, 't': t} this is for parse_data(x, label); see the third line in the parse_data function. 
    return {'x': x, 'e': e, 't': t}

def probe_infnan(v, name, extras={}):
    nps = torch.isnan(v)
    s = nps.sum().item()
    if s > 0:
        print('>>> {} >>>'.format(name))
        print(name, s)
        print(v[nps])
        for k, val in extras.items():
            print(k, val, val.sum().item())
        quit()


class Identity(nn.Module):
    def forward(self, *args):
        if len(args) == 1:
            return args[0]
        return args

def get_batcnnorm(bn, nr_features=None, nr_dims=1):
    if isinstance(bn, nn.Module):
        return bn

    assert 1 <= nr_dims <= 3

    if bn in (True, 'async'):
        clz_name = 'BatchNorm{}d'.format(nr_dims)
        return getattr(nn, clz_name)(nr_features)
    else:
        raise ValueError('Unknown type of batch normalization: {}.'.format(bn))


def get_dropout(dropout, nr_dims=1):
    if isinstance(dropout, nn.Module):
        return dropout

    if dropout is True:
        dropout = 0.5
    if nr_dims == 1:
        return nn.Dropout(dropout, True)
    else:
        clz_name = 'Dropout{}d'.format(nr_dims)
        return getattr(nn, clz_name)(dropout)


def get_activation(act):
    if isinstance(act, nn.Module):
        return act

    assert type(act) is str, 'Unknown type of activation: {}.'.format(act)
    act_lower = act.lower()
    if act_lower == 'identity':
        return Identity()
    elif act_lower == 'relu':
        return nn.ReLU(True)
    elif act_lower == 'selu':
        return nn.SELU(True)
    elif act_lower == 'sigmoid':
        return nn.Sigmoid()
    elif act_lower == 'tanh':
        return nn.Tanh()
    else:
        try:
            return getattr(nn, act)
        except AttributeError:
            raise ValueError('Unknown activation function: {}.'.format(act))


def get_optimizer(optimizer, model, *args, **kwargs):
    if isinstance(optimizer, (optim.Optimizer)):
        return optimizer

    if type(optimizer) is str:
        try:
            optimizer = getattr(optim, optimizer)
        except AttributeError:
            raise ValueError('Unknown optimizer type: {}.'.format(optimizer))
    return optimizer(filter(lambda p: p.requires_grad, model.parameters()), *args, **kwargs)
    

def stmap(func, iterable):
    if isinstance(iterable, six.string_types):
        return func(iterable)
    elif isinstance(iterable, (collections.Sequence, collections.UserList)):
        return [stmap(func, v) for v in iterable]
    elif isinstance(iterable, collections.Set):
        return {stmap(func, v) for v in iterable}
    elif isinstance(iterable, (collections.Mapping, collections.UserDict)):
        return {k: stmap(func, v) for k, v in iterable.items()}
    else:
        return func(iterable)


def _as_tensor(o):
    from torch.autograd import Variable
    if isinstance(o, SKIP_TYPES):
        return o
    if isinstance(o, Variable):
        return o
    if torch.is_tensor(o):
        return o
    return torch.from_numpy(np.array(o))


def as_tensor(obj):
    return stmap(_as_tensor, obj)


def _as_numpy(o):
    from torch.autograd import Variable
    if isinstance(o, SKIP_TYPES):
        return o
    if isinstance(o, Variable):
        o = o
    if torch.is_tensor(o):
        return o.cpu().numpy()
    return np.array(o)


def as_numpy(obj):
    return stmap(_as_numpy, obj)


def _as_float(o):
    if isinstance(o, SKIP_TYPES):
        return o
    if torch.is_tensor(o):
        return o.item()
    arr = as_numpy(o)
    assert arr.size == 1
    return float(arr)


def as_float(obj):
    return stmap(_as_float, obj)


def _as_cpu(o):
    from torch.autograd import Variable
    if isinstance(o, Variable) or torch.is_tensor(o):
        return o.cpu()
    return o


def as_cpu(obj):
    return stmap(_as_cpu, obj)


## For synthetic dataset creation
import math
from sklearn.datasets import make_moons
from scipy.stats import norm


# Create a simple dataset
def create_twomoon_dataset(n, p):
    relevant, y = make_moons(n_samples=n, shuffle=True, noise=0.1, random_state=None)
    print(y.shape)
    noise_vector = norm.rvs(loc=0, scale=1, size=[n,p-2])
    data = np.concatenate([relevant, noise_vector], axis=1)
    print(data.shape)
    return data, y


def create_sin_dataset(n,p):
    x1=5*(np.random.uniform(0,1,n)).reshape(-1,1)
    x2=5*(np.random.uniform(0,1,n)).reshape(-1,1)
    y=np.sin(x1)*np.cos(x2)**3
    relevant=np.hstack((x1,x2))
    noise_vector = norm.rvs(loc=0, scale=1, size=[n,p-2])
    data = np.concatenate([relevant, noise_vector], axis=1)
    return data, y.astype(np.float32)



def create_simple_sin_dataset(n, p):
    '''This dataset was added to provide an example of L1 norm reg failure for presentation.
    '''
    assert p == 2
    x1 = np.random.uniform(-math.pi, math.pi, n).reshape(n ,1)
    x2 = np.random.uniform(-math.pi, math.pi, n).reshape(n, 1)
    y = np.sin(x1)
    data = np.concatenate([x1, x2], axis=1)
    print("data.shape: {}".format(data.shape))
    return data, y
    

In [12]:
# 2. layers.py
import torch.nn as nn
import torch
import math

__all__ = [
    'LinearLayer', 'MLPLayer', 'FeatureSelector',
]

class FeatureSelector(nn.Module):
    def __init__(self, input_dim, sigma, device):
        super(FeatureSelector, self).__init__()
        self.mu = torch.nn.Parameter(0.01*torch.randn(input_dim, ), requires_grad=True)
        self.noise = torch.randn(self.mu.size()) 
        self.sigma = sigma
        self.device = device
    
    def forward(self, prev_x):
        z = self.mu + self.sigma*self.noise.normal_()*self.training 
        stochastic_gate = self.hard_sigmoid(z)
        new_x = prev_x * stochastic_gate
        return new_x
    
    def hard_sigmoid(self, x):
        return torch.clamp(x+0.5, 0.0, 1.0)

    def regularizer(self, x):
        ''' Gaussian CDF. '''
        return 0.5 * (1 + torch.erf(x / math.sqrt(2))) 

    def _apply(self, fn):
        super(FeatureSelector, self)._apply(fn)
        self.noise = fn(self.noise)
        return self

class LinearLayer(nn.Sequential):
    def __init__(self, in_features, out_features, batch_norm=None, dropout=None, bias=None, activation=None):
        if bias is None:
            bias = (batch_norm is None)

        modules = [nn.Linear(in_features, out_features, bias=bias)]
        if batch_norm is not None and batch_norm is not False:
            modules.append(get_batcnnorm(batch_norm, out_features, 1))
        if dropout is not None and dropout is not False:
            modules.append(get_dropout(dropout, 1))
        if activation is not None and activation is not False:
            modules.append(get_activation(activation))
        super().__init__(*modules)

    def reset_parameters(self):
        for module in self.modules():
            if isinstance(module, nn.Linear):
                module.reset_parameters()

class MLPLayer(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dims, batch_norm=None, dropout=None, activation='relu', flatten=True):
        super().__init__()

        if hidden_dims is None:
            hidden_dims = []
        elif type(hidden_dims) is int:
            hidden_dims = [hidden_dims]

        dims = [input_dim]
        dims.extend(hidden_dims)
        dims.append(output_dim)
        modules = []

        nr_hiddens = len(hidden_dims)
        for i in range(nr_hiddens):
            layer = LinearLayer(dims[i], dims[i+1], batch_norm=batch_norm, dropout=dropout, activation=activation)
            modules.append(layer)
        layer = nn.Linear(dims[-2], dims[-1], bias=True)
        modules.append(layer)
        self.mlp = nn.Sequential(*modules)
        self.flatten = flatten

    def reset_parameters(self):
        for module in self.modules():
            if isinstance(module, nn.Linear):
                module.reset_parameters()

    def forward(self, input):
        if self.flatten:
            input = input.view(input.size(0), -1)
        return self.mlp(input)

In [13]:
# models.py
import torch.nn as nn
import torch
import math
import numpy as np
from torch.autograd import Variable


# from .layers import MLPLayer, FeatureSelector, GatingLayer
# from .losses import PartialLogLikelihood

__all__ = ['MLPModel', 'MLPRegressionModel', 'MLPClassificationModel', 'LinearRegressionModel', 'LinearClassificationModel']


class ModelIOKeysMixin(object):
    def _get_input(self, feed_dict):
        return feed_dict['input']

    def _get_label(self, feed_dict):
        return feed_dict['label']

    def _get_covariate(self, feed_dict):
        '''For cox'''
        return feed_dict['X']

    def _get_fail_indicator(self, feed_dict):
        '''For cox'''
        return feed_dict['E'].reshape(-1, 1)

    def _get_failure_time(self, feed_dict):
        '''For cox'''
        return feed_dict['T']

    def _compose_output(self, value):
        return dict(pred=value)


class MLPModel(MLPLayer):
    def freeze_weights(self):
        for name, p in self.named_parameters():
            if name != 'mu':
                p.requires_grad = False

    def get_gates(self, mode):
        if mode == 'raw':
            return self.mu.detach().cpu().numpy()
        elif mode == 'prob':
            return np.minimum(1.0, np.maximum(0.0, self.mu.detach().cpu().numpy() + 0.5)) 
        else:
            raise NotImplementedError()
            

class STGClassificationModel(MLPModel, ModelIOKeysMixin):
    def __init__(self, input_dim, nr_classes, hidden_dims, device, batch_norm=None, dropout=None, activation='relu',
                 sigma=1.0, lam=0.1):
        super().__init__(input_dim, nr_classes, hidden_dims,
                         batch_norm=batch_norm, dropout=dropout, activation=activation)
        self.FeatureSelector = FeatureSelector(input_dim, sigma, device)
        self.softmax = nn.Softmax()
        self.loss = nn.CrossEntropyLoss()
        self.reg = self.FeatureSelector.regularizer
        self.lam = lam 
        self.mu = self.FeatureSelector.mu
        self.sigma = self.FeatureSelector.sigma
        
    def forward(self, feed_dict):
        x = self.FeatureSelector(self._get_input(feed_dict))
        logits = super().forward(x)
        if self.training:
            loss = self.loss(logits, self._get_label(feed_dict))
            reg = torch.mean(self.reg((self.mu + 0.5)/self.sigma)) 
            total_loss = loss + self.lam * reg 
            return total_loss, dict(), dict() 
        else:
            return self._compose_output(logits)

    def _compose_output(self, logits):
        value = self.softmax(logits)
        _, pred = value.max(dim=1)
        return dict(prob=value, pred=pred, logits=logits)

class MLPClassificationModel(MLPModel, ModelIOKeysMixin):
    def __init__(self, input_dim, nr_classes, hidden_dims, batch_norm=None, dropout=None, activation='relu'):
        super().__init__(input_dim, nr_classes, hidden_dims,
                         batch_norm=batch_norm, dropout=dropout, activation=activation)
        self.softmax = nn.Softmax()
        self.loss = nn.CrossEntropyLoss()

    def forward(self, feed_dict):
        logits = super().forward(self._get_input(feed_dict))
        if self.training:
            loss = self.loss(logits, self._get_label(feed_dict))
            return loss, dict(), dict()
        else:
            return self._compose_output(logits)

    def _compose_output(self, logits):
        value = self.softmax(logits)
        _, pred = value.max(dim=1)
        return dict(prob=value, pred=pred, logits=logits)

    

In [14]:
# meter.py
import six
import itertools
import collections
import json

def map_exec(func, *iterables):
    return list(map(func, *iterables))


class AverageMeter(object):
    """Computes and stores the average and current value"""
    val = 0
    avg = 0
    sum = 0
    count = 0
    tot_count = 0

    def __init__(self):
        self.reset()
        self.tot_count = 0

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.tot_count += n
        self.avg = self.sum / self.count


class GroupMeters(object):
    def __init__(self):
        self._meters = collections.defaultdict(AverageMeter)

    def reset(self):
        map_exec(AverageMeter.reset, self._meters.values())

    def update(self, updates=None, value=None, n=1, **kwargs):
        """
        Example:
            >>> meters.update(key, value)
            >>> meters.update({key1: value1, key2: value2})
            >>> meters.update(key1=value1, key2=value2)
        """
        if updates is None:
            updates = {}
        if updates is not None and value is not None:
            updates = {updates: value}
        updates.update(kwargs)
        for k, v in updates.items():
            self._meters[k].update(v, n=n)

    def __getitem__(self, name):
        return self._meters[name]

    def items(self):
        return self._meters.items()

    @property
    def sum(self):
        return {k: m.sum for k, m in self._meters.items() if m.count > 0}

    @property
    def avg(self):
        return {k: m.avg for k, m in self._meters.items() if m.count > 0}

    @property
    def val(self):
        return {k: m.val for k, m in self._meters.items() if m.count > 0}

    def format(self, caption, values, kv_format, glue):
        meters_kv = self._canonize_values(values)
        log_str = [caption]
        log_str.extend(itertools.starmap(kv_format.format, sorted(meters_kv.items())))
        return glue.join(log_str)

    def format_simple(self, caption, values='avg', compressed=True):
        if compressed:
            return self.format(caption, values, '{}={:4f}', ' ')
        else:
            return self.format(caption, values, '\t{} = {:4f}', '\n')

    def dump(self, filename, values='avg'):
        meters_kv = self._canonize_values(values)
        with open(filename, 'a') as f:
            #f.write(io.dumps_json(meters_kv, compressed=False))
            f.write(json.dumps(meters_kv, cls=JsonObjectEncoder, sort_keys=True, indent=4, separators=(',', ': ')))
            f.write('\n')

    def _canonize_values(self, values):
        if isinstance(values, six.string_types):
            assert values in ('avg', 'val', 'sum')
            meters_kv = getattr(self, values)
        else:
            meters_kv = values
        return meters_kv


In [15]:
# stg.py
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import logging.config 
import os.path as osp
import time
import numpy as np
import logging
logger = logging.getLogger("my-logger")


__all__ = ['STG']


def _standard_truncnorm_sample(lower_bound, upper_bound, sample_shape=torch.Size()):
    r"""
    Implements accept-reject algorithm for doubly truncated standard normal distribution.
    (Section 2.2. Two-sided truncated normal distribution in [1])
    [1] Robert, Christian P. "Simulation of truncated normal variables." Statistics and computing 5.2 (1995): 121-125.
    Available online: https://arxiv.org/abs/0907.4010
    Args:
        lower_bound (Tensor): lower bound for standard normal distribution. Best to keep it greater than -4.0 for
        stable results
        upper_bound (Tensor): upper bound for standard normal distribution. Best to keep it smaller than 4.0 for
        stable results
    """
    x = torch.randn(sample_shape)
    done = torch.zeros(sample_shape).byte() 
    while not done.all():
        proposed_x = lower_bound + torch.rand(sample_shape) * (upper_bound - lower_bound)
        if (upper_bound * lower_bound).lt(0.0):  # of opposite sign
            log_prob_accept = -0.5 * proposed_x**2
        elif upper_bound < 0.0:  # both negative
            log_prob_accept = 0.5 * (upper_bound**2 - proposed_x**2)
        else:  # both positive
            assert(lower_bound.gt(0.0))
            log_prob_accept = 0.5 * (lower_bound**2 - proposed_x**2)
        prob_accept = torch.exp(log_prob_accept).clamp_(0.0, 1.0)
        accept = torch.bernoulli(prob_accept).byte() & ~done
        if accept.any():
            accept = accept.bool()
            x[accept] = proposed_x[accept]
            accept = accept.byte()
            done |= accept
    return x


class STG(object):
    def __init__(self, device, input_dim=784, output_dim=10, hidden_dims=[400, 200], activation='relu', sigma=0.5, lam=0.1,
                optimizer='Adam', learning_rate=1e-5,  batch_size=100, freeze_onward=None, feature_selection=True, weight_decay=1e-3, 
                task_type='classification', report_maps=False, random_state=1, extra_args=None):
        self.batch_size = batch_size
        self.activation = activation
        self.device = self.get_device(device)
        self.report_maps = report_maps 
        self.task_type = task_type
        self.extra_args = extra_args
        self.freeze_onward = freeze_onward
        self._model = self.build_model(input_dim, output_dim, hidden_dims, activation, sigma, lam, 
                                       task_type, feature_selection)
        self._model.apply(self.init_weights)
        self._model = self._model.to(device)
        self._optimizer = get_optimizer(optimizer, self._model, lr=learning_rate, weight_decay=weight_decay)
    
    def get_device(self, device):
        if device == "cpu":
            device = torch.device("cpu")
        elif device == None:
            args_cuda = torch.cuda.is_available()
            device = device = torch.device("cuda" if args_cuda else "cpu")
        else:
            raise NotImplementedError("Only 'cpu' or 'cuda' is a valid option.")
        return device
        
        
    def init_weights(self, m):
        if isinstance(m, nn.Linear):
            stddev = torch.tensor(0.1)
            shape = m.weight.shape
            m.weight = nn.Parameter(_standard_truncnorm_sample(lower_bound=-2*stddev, upper_bound=2*stddev, 
                                  sample_shape=shape))
            torch.nn.init.zeros_(m.bias)

    def build_model(self, input_dim, output_dim, hidden_dims, activation, sigma, lam, task_type, feature_selection):
        if task_type == 'classification':
            self.metric = nn.CrossEntropyLoss()
            self.tensor_names = ('input','label')
            return STGClassificationModel(input_dim, output_dim, hidden_dims, device=self.device, activation=activation, sigma=sigma, lam=lam)
        if task_type == "reconstruction":
            self.metric = nn.MSELoss()
            self.tensor_names = ('input','label')
            return STGReconstructionModel(input_dim, output_dim, hidden_dims, device=self.device, activation=activation, sigma=sigma, lam=lam)
        else:
            raise NotImplementedError()

    def train_step(self, feed_dict, meters=None):
        assert self._model.training

        loss, logits, monitors = self._model(feed_dict)
        self._optimizer.zero_grad()
        loss.backward()
        self._optimizer.step()
        #probe_infnan(logits, 'logits')
        if self.task_type=='cox':
            ci = calc_concordance_index(logits.detach().numpy(), 
                    feed_dict['E'].detach().numpy(), feed_dict['T'].detach().numpy())
        if self.extra_args=='l1-softthresh':
            self._model.mlp[0][0].weight.data = self._model.prox_op(self._model.mlp[0][0].weight)

        loss = as_float(loss)
        if meters is not None:
            meters.update(loss=loss)
            if self.task_type =='cox':
                meters.update(CI=ci)
            meters.update(monitors)

    def get_dataloader(self, X, y, shuffle):
        if self.task_type == 'classification' or self.task_type == 'reconstruction':
            data_loader = FastTensorDataLoader(torch.from_numpy(X).float().to(self.device), 
                        torch.from_numpy(y).long().to(self.device), tensor_names=self.tensor_names,
                        batch_size=self.batch_size, shuffle=shuffle)
        else:
            raise NotImplementedError()

        return data_loader 

    def fit(self, X, y, nr_epochs, valid_X=None, valid_y=None, 
        verbose=True, meters=None, early_stop=None, print_interval=1, shuffle=False):
        data_loader = self.get_dataloader(X, y, shuffle)

        if valid_X is not None:
            val_data_loader = self.get_dataloader(valid_X, valid_y, shuffle)
        else:
            val_data_loader = None
        self.train(data_loader, nr_epochs, val_data_loader, verbose, meters, early_stop, print_interval)

    def evaluate(self, X, y):
        data_loader = self.get_dataloader(X, y, shuffle=None)
        meters = GroupMeters()
        self.validate(data_loader, self.metric, meters, mode='test')
        print(meters.format_simple(''))

    def predict(self, X, verbose=True):
        dataset = SimpleDataset(X)
        data_loader = DataLoader(dataset, batch_size=X.shape[0], shuffle=False) 
        res = []
        self._model.eval()
        for feed_dict in data_loader:
            feed_dict_np = as_numpy(feed_dict)
            feed_dict = as_tensor(feed_dict)
            with torch.no_grad():
                output_dict = self._model(feed_dict)
            output_dict_np = as_numpy(output_dict)
            res.append(output_dict_np['pred'])
        return np.concatenate(res, axis=0)

    def train_epoch(self, data_loader, meters=None):
        if meters is None:
            meters = GroupMeters()

        self._model.train()
        end = time.time()
        for feed_dict in data_loader:
            data_time = time.time() - end; end = time.time()
            self.train_step(feed_dict, meters=meters)
            step_time = time.time() - end; end = time.time()
            #if dev:
            #meters.update({'time/data': data_time, 'time/step': step_time})
        return meters

    def train(self, data_loader, nr_epochs, val_data_loader=None, verbose=True, 
        meters=None, early_stop=None, print_interval=1):
        if meters is None:
            meters = GroupMeters()

        for epoch in range(1, 1 + nr_epochs):
            meters.reset()
            if epoch == self.freeze_onward:
                self._model.freeze_weights()
            self.train_epoch(data_loader, meters=meters)
            if verbose and epoch % print_interval == 0:
                self.validate(val_data_loader, self.metric, meters)
                caption = 'Epoch: {}:'.format(epoch)
                print(meters.format_simple(caption))
            if early_stop is not None:
                flag = early_stop(self._model)
                if flag:
                    break

    def validate_step(self, feed_dict, metric, meters=None, mode='valid'):
        with torch.no_grad():
            pred = self._model(feed_dict)
        if self.task_type == 'classification':
            result = metric(pred['logits'], self._model._get_label(feed_dict))
        else:
            raise NotImplementedError()

        if meters is not None:
            meters.update({mode+'_loss':result})
            if self.task_type=='cox':
                meters.update({mode+'_CI':val_CI})

    def validate(self, data_loader, metric, meters=None, mode='valid'):
        if meters is None:
            meters = GroupMeters()

        self._model.eval()
        end = time.time()
        for fd in data_loader:
            data_time = time.time() - end; end = time.time()
            self.validate_step(fd, metric, meters=meters, mode=mode)
            step_time = time.time() - end; end = time.time()

        return meters.avg

    def get_gates(self, mode):
        return self._model.get_gates(mode)



In [16]:
args_cuda = torch.cuda.is_available()
device = "cuda" if args_cuda else "cpu"
feature_selection = True
model = STG(task_type='classification',input_dim=X_train.shape[1], output_dim=2, hidden_dims=[200, 60], activation='relu',
    optimizer='Adam', learning_rate=0.01, batch_size=X_train.shape[0], feature_selection=feature_selection, sigma=0.5, lam=0.5, random_state=1, device=device) 


In [17]:
model.fit(X_train.to_numpy(), y_train.to_numpy(), nr_epochs=1000, valid_X=X_valid.to_numpy(), valid_y=y_valid.to_numpy(), print_interval=200)




/Users/mamun/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:215: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
/Users/mamun/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:217: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
/Users/mamun/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:219: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
/Users/mamun/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:78: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 200: loss=0.342700 valid_loss=0.002340
Epoch: 400: loss=0.327726 valid_loss=0.032171
Epoch: 600: loss=0.320385 valid_loss=0.248116
Epoch: 800: loss=0.309509 valid_loss=0.044702
Epoch: 1000: loss=0.310075 valid_loss=0.014132


In [18]:
prob = model.get_gates(mode = 'prob')
prob_filtered = []
for i, x in enumerate(prob):
    prob_filtered.append((i, x))

    
prob_filtered.sort(key = lambda x: x[1], reverse=True)
print(len(prob_filtered))
print(*prob_filtered, sep = "\n")

1022
(43, 1.0)
(317, 1.0)
(385, 1.0)
(509, 1.0)
(512, 1.0)
(620, 1.0)
(628, 1.0)
(663, 1.0)
(766, 1.0)
(262, 0.9686836)
(490, 0.95932406)
(866, 0.87786764)
(73, 0.8730328)
(716, 0.86529917)
(436, 0.84358597)
(664, 0.810651)
(633, 0.7933525)
(922, 0.7923891)
(960, 0.7917078)
(99, 0.7598001)
(890, 0.7498144)
(410, 0.7340355)
(77, 0.7004177)
(26, 0.69604516)
(1001, 0.65572035)
(399, 0.6446345)
(964, 0.58424264)
(672, 0.5591872)
(38, 0.54006386)
(774, 0.53479934)
(962, 0.5325687)
(465, 0.5055366)
(54, 0.50200444)
(417, 0.49159747)
(454, 0.49098492)
(703, 0.48926735)
(397, 0.4850421)
(982, 0.47453162)
(941, 0.47258866)
(411, 0.46717504)
(673, 0.45691586)
(837, 0.45154312)
(169, 0.44324625)
(1013, 0.43830755)
(215, 0.4356114)
(190, 0.4222411)
(370, 0.41159254)
(714, 0.39193904)
(113, 0.38729998)
(200, 0.38551196)
(697, 0.383628)
(625, 0.3786059)
(81, 0.3716334)
(969, 0.36709508)
(278, 0.36585626)
(311, 0.36296427)
(282, 0.36209983)
(63, 0.35531992)
(510, 0.35415483)
(452, 0.3511464)
(297, 0.

In [19]:
raw = model.get_gates(mode = 'raw')
raw_filtered = []
for i, x in enumerate(raw):
    if x>0: raw_filtered.append((i, x))

raw_filtered.sort(key = lambda x: x[1], reverse=True)
print(len(raw_filtered))
print(*raw_filtered, sep="\n")

33
(43, 0.8595033)
(317, 0.7928253)
(385, 0.7583463)
(512, 0.7189029)
(620, 0.6643486)
(766, 0.6528963)
(509, 0.5164768)
(663, 0.5105725)
(628, 0.5030268)
(262, 0.46868357)
(490, 0.45932406)
(866, 0.37786764)
(73, 0.37303284)
(716, 0.36529917)
(436, 0.34358597)
(664, 0.310651)
(633, 0.2933525)
(922, 0.2923891)
(960, 0.2917078)
(99, 0.2598001)
(890, 0.24981438)
(410, 0.23403552)
(77, 0.20041768)
(26, 0.19604516)
(1001, 0.15572034)
(399, 0.1446345)
(964, 0.084242664)
(672, 0.05918716)
(38, 0.04006384)
(774, 0.034799322)
(962, 0.032568675)
(465, 0.0055365954)
(54, 0.002004441)


In [24]:
y_pred = model.predict(X.to_numpy())

/Users/mamun/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:78: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [25]:
y_pred[100:110]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])